In [1]:
import numpy as np

In [2]:
class Playground:
    
    def __init__(self, height, width, alpha = 0.2):
        
        assert (alpha < 1) and (alpha > 0)
        assert (height >= 3) and (type(height) == int)
        assert (width >= 3) and (type(width) == int)
        
        self.height = height
        self.width = width
        self.square = self.height * self.width
        self.alpha = alpha
        
        typical_shape = (self.height, self.width)
        
        self.mines = np.zeros(typical_shape, dtype = 'bool')
        self.place_mines()
        
        self.values = np.zeros(typical_shape)
        self.find_values()
        
        self.player_field = np.full(typical_shape, '*')
        
    def place_mines(self):
        self.amount_of_mines = int(np.round(self.square * self.alpha))
        positions = np.arange(self.square)
        np.random.shuffle(positions)
        for pos in positions[:self.amount_of_mines]:
            self.mines[pos // self.width, pos % self.width] = True
            
    def find_values(self):
        a = list(range(-1, 2))
        directions = np.array(np.meshgrid(a, a)).T.reshape(-1, 2)
        self.directions = np.delete(directions, 4, 0)
        for y in range(self.height):
            for x in range(self.width):
                
                current_value = 0
                
                if self.mines[y, x]:
                    self.values[y, x] = -1
                    continue
                
                banned_idx = self.banner(x, y)            
                current_directions = np.delete(self.directions, banned_idx, 0)
                
                for direction in current_directions:
                    if self.mines[y + direction[0], x + direction[1]]:
                        current_value += 1
                        
                self.values[y, x] = current_value
                
    
    def banner(self, x, y):
        out = []
        
        if y == 0:
            for idx, i in enumerate(self.directions):
                if i[0] == -1:
                    out.append(idx)
                            
        if x == 0:
            for idx, i in enumerate(self.directions):
                if i[1] == -1:
                    out.append(idx)
                            
        if y == self.height - 1:
            for idx, i in enumerate(self.directions):
                if i[0] == 1:
                    out.append(idx)
                            
        if x == self.width - 1:
            for idx, i in enumerate(self.directions):
                if i[1] == 1:
                    out.append(idx)
        
        return out
    
    def show(self):
        print()
        for y in range(self.height):
            for x in range(self.width):
                print(self.player_field[y, x], end="")
            print()
        print()
        
    def enter(self, x, y):
        
        result = 0
        
        if self.player_field[y, x] == '*':
            if self.mines[y, x]:
                self.player_field[y, x] = '!'
                return -1
            
            result += 1
            self.player_field[y, x] = str(self.values[y, x])
            
            if self.values[y, x] == 0:
                if x > 0: result += self.enter(x-1,y)
                if y > 0: result += self.enter(x,y-1)
                if x < self.width - 1: result += self.enter(x+1,y)
                if y < self.height - 1: result += self.enter(x,y+1)
            
        return result

In [3]:
class game:
    
    def __init__(self, height, width, alpha = 0.2):
        self.height = height
        self.width = width
        self.alpha = alpha
        
    def start(self):
        self.pg = Playground(self.height, self.width, self.alpha)
        self.mines_amount = self.pg.amount_of_mines
        self.closed = self.height * self.width
        self.game_runned = True
    
    def do_step(self, x, y):
        if self.game_runned:
            result = self.pg.enter(int(x), int(y))
            if result == -1:
                self.fail_game()
            else:
                self.closed -= result
        self.pg.show()
        if self.closed == self.mines_amount:
            self.win_game()
        
    def fail_game(self):
        self.game_runned = False
        print("You have failed!")
        
    def win_game(self):
        self.game_runned = False
        print("You have won!")
        
    def show(self):
        self.pg.show()

In [4]:
trial = game(4, 3, 0.1)

In [5]:
trial.start()
trial.show()
while(trial.game_runned):
    trial.do_step(input("Enter x:\t"), input("Enter y:\t"))


***
***
***
***

Enter x:	0
Enter y:	0

000
000
111
***

Enter x:	0
Enter y:	3

000
000
111
1**

Enter x:	2
Enter y:	3

000
000
111
1*1

You have won!
